In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
! pip install transformers==3.5.1

     |████████████████████████████████| 1.3MB 15.6MB/s 
     |████████████████████████████████| 890kB 48.2MB/s 
     |████████████████████████████████| 2.9MB 49.8MB/s 
     |████████████████████████████████| 1.1MB 41.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=fa41c2665039df3820bd1987a38d2f8c247066a74933b4af0fd286d9d2e59c95
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [3]:
%pip install tokenizers -U

     |████████████████████████████████| 2.9MB 16.5MB/s 
ERROR: transformers 3.5.1 has requirement tokenizers==0.9.3, but you'll have tokenizers 0.9.4 which is incompatible.
  Found existing installation: tokenizers 0.9.3
    Uninstalling tokenizers-0.9.3:
      Successfully uninstalled tokenizers-0.9.3


In [2]:
!pip install simpletransformers

In [3]:
!pip install datasets

In [4]:
from datasets import load_dataset
dataset = load_dataset("wikisql")
print(dataset['train'][0])

Using custom data configuration default


Dataset wiki_sql downloaded and prepared to /root/.cache/huggingface/datasets/wiki_sql/default/0.1.0/2e98053891fd8f9b2c4348bba609ce40cc0a4d7f621191cebcd7cb558b5f8a70. Subsequent calls will reuse this data.
{'phase': 1, 'question': 'Tell me what the notes are for South Australia ', 'sql': {'agg': 0, 'conds': {'column_index': [3], 'condition': ['SOUTH AUSTRALIA'], 'operator_index': [0]}, 'human_readable': 'SELECT Notes FROM table WHERE Current slogan = SOUTH AUSTRALIA', 'sel': 5}, 'table': {'caption': '', 'header': ['State/territory', 'Text/background colour', 'Format', 'Current slogan', 'Current series', 'Notes'], 'id': '1-1000181-1', 'name': 'table_1000181_1', 'page_id': '', 'page_title': '', 'rows': [['Australian Capital Territory', 'blue/white', 'Yaa·nna', 'ACT · CELEBRATION OF A CENTURY 2013', 'YIL·00A', 'Slogan screenprinted on plate'], ['New South Wales', 'black/yellow', 'aa·nn·aa', 'NEW SOUTH WALES', 'BX·99·HI', 'No slogan on current series'], ['New South Wales', 'black/white', '

In [5]:
import sqlite3

conn = sqlite3.connect('temp.db')
print("Opened database successfully");

conn.execute('''
CREATE TABLE IF NOT EXISTS team_data(team text, 
                      country text, 
                      season integer, 
                      total_goals integer);''')

conn.commit()

print("Table created successfully");

conn.execute('''DROP TABLE team_data''')

print("Table deleted successfully");

conn.close()

Opened database successfully
Table created successfully
Table deleted successfully


In [6]:
import re
import sklearn
import numpy as np
import pandas as pd
import sys

In [7]:
PEN_FACTOR = 3

In [8]:
def check_query(data, pred_query):
    penalty = 1.0
    conn = sqlite3.connect('temp.db')
    inp, query = data['question'], data['sql']['human_readable']
    col_names, col_types = data['table']['header'], data['table']['types']
    conditions = data['sql']['conds']['condition']
    S = """create table test( """
    res = ""
    for i in range(len(col_names)):
        ch_new = re.sub(r'[^a-zA-Z0-9]','', col_names[i])
        if(i < len(col_names)-1):
            res += ch_new + " " + col_types[i] + " , "
        else:
            res += ch_new + " " + col_types[i]
    Q = S + res + ")"
    print(Q)
    conn.execute(Q)
    for col in col_names:
        col_new = re.sub(r'[^a-zA-Z0-9]','', col)
        pred_query = pred_query.replace(col, col_new)
    for cond in conditions:
        if(not cond.isnumeric()):
            pred_query = pred_query.replace(cond, "'"+cond+"'")
    pred_query = pred_query.replace('table', 'test')
    print(pred_query)
    try:
        conn.execute(pred_query)
        conn.commit()
    except:
        penalty *= PEN_FACTOR
        print("QUERY INCORRECT")
    conn.execute("drop table test")
    conn.close()
    return penalty

In [9]:
S = """SELECT Notes FROM table WHERE Current slogan = SOUTH AUSTRALIA"""
check_query(dataset['train'][0], S)

create table test( Stateterritory text , Textbackgroundcolour text , Format text , Currentslogan text , Currentseries text , Notes text)
SELECT Notes FROM test WHERE Currentslogan = 'SOUTH AUSTRALIA'


1.0

In [10]:
train = []
for d in dataset["train"]:
    temp = []
    temp.append(d['question'])
    temp.append(d["sql"]["human_readable"])
    train.append(temp)

validation = []
for d in dataset["validation"]:
    temp = []
    temp.append(d['question'])
    temp.append(d["sql"]["human_readable"])
    validation.append(temp)

In [11]:
import logging

import pandas as pd
from simpletransformers.seq2seq import Seq2SeqModel

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [12]:
train_df = pd.DataFrame(train, columns=["input_text", "target_text"])

eval_df = pd.DataFrame(validation, columns=["input_text", "target_text"])

model_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "max_seq_length": 50,
    "train_batch_size": 32,
    "num_train_epochs": 20,
    "save_eval_checkpoints": False,
    "save_model_every_epoch": False,
    "evaluate_generated_text": True,
    "evaluate_during_training_verbose": True,
    "use_multiprocessing": False,
    "max_length": 25,
    "manual_seed": 4,
    "output_dir": '/content/drive/MyDrive/outputs'
}
encoder_type = "distilbert"

model = Seq2SeqModel(
    encoder_decoder_type="bart",
    encoder_decoder_name="/content/drive/MyDrive/outputs/checkpoint-30000",
    args=model_args,
    use_cuda=True
)

In [13]:
def scaled_nll(true_labels, model_preds):
    loss = sklearn.metrics.log_loss(true_labels, model_preds)
    pen = []
    N = model_preds.size(0)
    for i in range(N):
      probs = model_preds[i,:]
      max_idxs = torch.argmax(probs, dim = -1)
      pen.append(check_query(true_labels, max_ids))
    pen = torch.Tensor(pen)
    loss *= pen
    return torch.mean(loss)

In [14]:
model.predict(
        [
            "SELECT Notes FROM table WHERE Current slogan = SOUTH AUSTRALIA",
            "SELECT Notes FROM table WHERE Current slogan = NORTH AUSTRALIA"
        ]
)

['SELECT Notes FROM table WHERE Current slogan = SOUTH AUSTRALIA',
 'SELECT Notes FROM table WHERE Current slogan = north australia']

In [15]:
model.train_model(train_df, nll = scaled_nll)

INFO:simpletransformers.seq2seq.seq2seq_utils: Creating features from dataset file at cache_dir/


INFO:simpletransformers.seq2seq.seq2seq_model: Training started


KeyboardInterrupt: ignored